In [27]:
import os
import shutil
from pathlib import Path
from dotenv import load_dotenv
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.face import FaceClient
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials
from collections import defaultdict
import hashlib
import time

In [28]:
load_dotenv()

# Load all available credentials
api_key = os.getenv('API_KEY')
endpoint = os.getenv('URL')
api_key_face = os.getenv('API_KEY_FACE')
endpoint_face = os.getenv('URL_FACE')

print("Available credentials:")
print(f"  API_KEY: {'✓' if api_key else '✗'}")
print(f"  URL: {endpoint if endpoint else '✗'}")
print(f"  API_KEY_FACE: {'✓' if api_key_face else '✗'}")
print(f"  URL_FACE: {endpoint_face if endpoint_face else '✗'}")
print()

# Try Computer Vision first
cv_client = None
if api_key and endpoint:
    try:
        cv_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(api_key))
        print("✓ Computer Vision client created (for celebrities & face detection)")
    except Exception as e:
        print(f"✗ Computer Vision failed: {e}")

# Try Face API
face_client = None
if api_key_face and endpoint_face:
    try:
        face_client = FaceClient(endpoint_face, CognitiveServicesCredentials(api_key_face))
        print("✓ Face API client created (for detailed face detection)")
    except Exception as e:
        print(f"✗ Face API failed: {e}")

if not cv_client and not face_client:
    print("\n❌ ERROR: No working clients! Check your .env file")
else:
    print("\n✅ Ready to process images!")

Available credentials:
  API_KEY: ✓
  URL: https://20231721-7105-resource.cognitiveservices.azure.com/
  API_KEY_FACE: ✓
  URL_FACE: https://facefrance.cognitiveservices.azure.com/

✓ Computer Vision client created (for celebrities & face detection)
✓ Face API client created (for detailed face detection)

✅ Ready to process images!


In [29]:

SOURCE_FOLDER = "People_Input"         
DESTINATION_FOLDER = "People_Output"  

# Create folders
Path(SOURCE_FOLDER).mkdir(exist_ok=True)
Path(DESTINATION_FOLDER).mkdir(exist_ok=True)

print(f"✓ Source folder: {SOURCE_FOLDER}")
print(f"✓ Destination folder: {DESTINATION_FOLDER}")

✓ Source folder: People_Input
✓ Destination folder: People_Output


In [30]:
def analyze_image(image_path):
    """Analyze image using available APIs"""
    result = {
        'celebrities': [],
        'faces': [],
        'face_details': []
    }
    
    # Try Computer Vision for celebrities and basic faces
    if cv_client:
        try:
            with open(image_path, 'rb') as image:
                analysis = cv_client.analyze_image_in_stream(
                    image,
                    visual_features=[
                        VisualFeatureTypes.faces,
                        VisualFeatureTypes.categories
                    ]
                )
                
                # Get celebrities
                for category in analysis.categories:
                    if category.detail and category.detail.celebrities:
                        for celeb in category.detail.celebrities:
                            if celeb.confidence > 0.6:
                                result['celebrities'].append({
                                    'name': celeb.name,
                                    'confidence': celeb.confidence
                                })
                
                # Get basic faces
                result['faces'] = analysis.faces
                
        except Exception as e:
            print(f"  Computer Vision error: {e}")
    
    # Try Face API for detailed face detection
    if face_client:
        try:
            with open(image_path, 'rb') as image:
                detected_faces = face_client.face.detect_with_stream(
                    image=image,
                    return_face_id=True,
                    return_face_attributes=['age', 'gender', 'emotion'],
                    recognition_model='recognition_04',
                    detection_model='detection_03'
                )
                result['face_details'] = detected_faces
                
        except Exception as e:
            print(f"  Face API error: {e}")
    
    return result

print("✓ Analysis function ready!")


✓ Analysis function ready!


In [31]:
# Statistics
stats = {
    'photos_processed': 0,
    'total_faces': 0,
    'unique_faces': set(),
    'face_counts': defaultdict(int)
}

# Get all image files
image_extensions = {'.jpg', '.jpeg', '.png', '.bmp', '.gif'}
source_path = Path(SOURCE_FOLDER)
image_files = [f for f in source_path.iterdir() 
               if f.is_file() and f.suffix.lower() in image_extensions]

print(f"Found {len(image_files)} images to process\n")
print("="*60)

dest_path = Path(DESTINATION_FOLDER)

# Process each image
for image_file in image_files:
    stats['photos_processed'] += 1
    print(f"\n📸 Processing: {image_file.name}")
    
    # Analyze image
    result = analyze_image(str(image_file))
    
    # Count total faces
    total_faces = (
        len(result['celebrities']) + 
        len(result['faces']) + 
        len(result['face_details'])
    )
    
    # No faces detected
    if total_faces == 0:
        no_face_folder = dest_path / "No_Faces_Detected"
        no_face_folder.mkdir(exist_ok=True)
        shutil.copy2(image_file, no_face_folder / image_file.name)
        print("  → No faces detected")
        continue
    
    folders_for_this_image = set()
    
    # Process celebrities
    for celeb in result['celebrities']:
        celeb_name = celeb['name'].replace(' ', '_').replace('/', '_')
        confidence = celeb['confidence']
        print(f"  ⭐ Celebrity: {celeb['name']} (confidence: {confidence:.2%})")
        
        folder_name = f"Celebrity_{celeb_name}"
        folders_for_this_image.add(folder_name)
        stats['unique_faces'].add(f"celeb_{celeb_name}")
    
    # Process Face API detailed faces (most accurate)
    if result['face_details']:
        for i, face in enumerate(result['face_details']):
            face_id = str(face.face_id)
            face_hash = hashlib.md5(face_id.encode()).hexdigest()[:8]
            
            # Get attributes if available
            age = face.face_attributes.age if face.face_attributes else "unknown"
            gender = face.face_attributes.gender if face.face_attributes else "unknown"
            
            folder_name = f"Person_{face_hash}"
            print(f"  👤 Face: {gender}, ~{age}yo → {folder_name}")
            
            folders_for_this_image.add(folder_name)
            stats['unique_faces'].add(face_hash)
    
    # Process Computer Vision faces (if Face API didn't work)
    elif result['faces']:
        for i, face in enumerate(result['faces']):
            face_properties = f"{face.age}_{face.gender}_{face.face_rectangle.left}_{face.face_rectangle.top}"
            face_hash = hashlib.md5(face_properties.encode()).hexdigest()[:8]
            
            folder_name = f"Person_{face_hash}"
            print(f"  👤 Face: {face.gender}, ~{face.age}yo → {folder_name}")
            
            folders_for_this_image.add(folder_name)
            stats['unique_faces'].add(face_hash)
    
    # Update stats
    stats['total_faces'] += len(folders_for_this_image)
    
    # Copy image to all relevant folders
    for folder_name in folders_for_this_image:
        face_folder = dest_path / folder_name
        face_folder.mkdir(exist_ok=True)
        shutil.copy2(image_file, face_folder / image_file.name)
        stats['face_counts'][folder_name] += 1
    
    # Small delay to avoid rate limiting
    time.sleep(0.5)

print("\n" + "="*60)
print("✓ Processing complete!")


Found 4 images to process


📸 Processing: gonca3.jpg
  Face API error: (InvalidRequest) Invalid request has been sent.
  👤 Face: None, ~Noneyo → Person_feaad53e
  Face API error: (InvalidRequest) Invalid request has been sent.
  👤 Face: None, ~Noneyo → Person_feaad53e

📸 Processing: mada1.jpg

📸 Processing: mada1.jpg
  Face API error: (InvalidRequest) Invalid request has been sent.
  👤 Face: None, ~Noneyo → Person_eb6e3539
  Face API error: (InvalidRequest) Invalid request has been sent.
  👤 Face: None, ~Noneyo → Person_eb6e3539

📸 Processing: mada2.jpg

📸 Processing: mada2.jpg
  Face API error: (InvalidRequest) Invalid request has been sent.
  👤 Face: None, ~Noneyo → Person_17907457
  Face API error: (InvalidRequest) Invalid request has been sent.
  👤 Face: None, ~Noneyo → Person_17907457

📸 Processing: photo.jpg

📸 Processing: photo.jpg
  Face API error: (InvalidRequest) Invalid request has been sent.
  👤 Face: None, ~Noneyo → Person_3f7d2c72
  Face API error: (InvalidRequest) Invali

In [32]:
print("\n" + "="*60)
print("PROCESSING SUMMARY")
print("="*60)
print(f"📸 Number of photos processed: {stats['photos_processed']}")
print(f"👤 Number of unique faces detected: {len(stats['unique_faces'])}")
print(f"👥 Total number of faces detected: {stats['total_faces']}")
print("\n📊 Faces detected (folder name : count):")
for name, count in sorted(stats['face_counts'].items()):
    print(f"   • {name}: {count}")
print("="*60)

# Show folder structure
print("\n📁 Folder structure created:")
organized_folders = sorted([f.name for f in dest_path.iterdir() if f.is_dir()])
for folder in organized_folders:
    file_count = len(list((dest_path / folder).iterdir()))
    print(f"   {folder}/ ({file_count} photos)")
print("="*60)


PROCESSING SUMMARY
📸 Number of photos processed: 4
👤 Number of unique faces detected: 4
👥 Total number of faces detected: 4

📊 Faces detected (folder name : count):
   • Person_17907457: 1
   • Person_3f7d2c72: 1
   • Person_eb6e3539: 1
   • Person_feaad53e: 1

📁 Folder structure created:
   No Faces Detected/ (0 photos)
   Person_17907457/ (1 photos)
   Person_3f7d2c72/ (1 photos)
   Person_eb6e3539/ (1 photos)
   Person_feaad53e/ (1 photos)


In [36]:
person_group_id = "known_faces"

# Attempt to delete old group for a clean start (optional)
try:
    face_client.person_group.delete(person_group_id)
    print("Deleted existing PersonGroup for clean slate")
except Exception:
    pass  # Ignore if not exists

try:
# Create PersonGroup
    face_client.person_group.create(
        person_group_id=person_group_id,
        name="Known Faces",
        recognition_model="recognition_04"
    )
    print("PersonGroup created")
except Exception as e:
    print(f"Error creating PersonGroup: {e}")

try:
# Create a person in that group
    person = face_client.person_group_person.create(person_group_id, "Goncalo")
    print(f"Person created: {person.name} (ID: {person.person_id})")
except Exception as e:
    print(f"Error creating person: {e}")

Error creating PersonGroup: (InvalidRequest) Invalid request has been sent.
Error creating person: (InvalidRequest) Invalid request has been sent.
